In [3]:
import sklearn
import numpy as np
import pandas as pd

import torch
cuda_available = torch.cuda.is_available()

from simpletransformers.ner import NERModel, NERArgs

from nervaluate import Evaluator

In [4]:
model_name='bert-base-multilingual-cased'

In [5]:
train_df =pd.read_csv("..\..\..\Dataset\\simpletransformers\\train.csv")
eval_df =pd.read_csv("..\..\..\Dataset\\simpletransformers\\val.csv")
test_df =pd.read_csv("..\..\..\Dataset\\simpletransformers\\test.csv")

In [6]:
label_list = [
    "O", 
    "B-PER", "I-PER",
    "B-ORG", "I-ORG",
    "B-LOC", "I-LOC",
    "B-PROD", "I-PROD",
    "B-WA", "I-WA",
    "B-EV", "I-EV",
]

In [7]:
# Create a NERModel
model_args=NERArgs()
# model_args.train_batch_size=64
# model_args.overwrite_output_dir=True
model_args = {
    "output_dir": "outputs/",
    "cache_dir": "cache_dir/",

    "fp16": True,
    "fp16_opt_level": "O1",
    "max_seq_length": 512,
    "train_batch_size": 8,
    "gradient_accumulation_steps": 1,
    "eval_batch_size": 8,
    # "num_train_epochs": 1,
    "num_train_epochs": 5,
    "weight_decay": 0,
    # "learning_rate": 4e-5,
    "learning_rate": 0.00008452,

    "adam_epsilon": 1e-8,
    "warmup_ratio": 0.06,
    "warmup_steps": 0,
    "max_grad_norm": 1.0,

    "logging_steps": 50,
    "save_steps": 2000,

    "overwrite_output_dir": True,
    "reprocess_input_data": False,
    "evaluate_during_training": True,

    "do_lower_case": False, #True if using uncased models
}


In [6]:
# model = NERModel('bert', 
#                 'outputs/checkpoint-2000',
#                 args=model_args,
#                 labels=label_list,
#                 use_cuda=cuda_available,
#                 )

In [8]:
model = NERModel('bert',
                model_name,
                args=model_args,
                labels=label_list,
                use_cuda=cuda_available,
                )

model.train_model(train_df,  eval_data=eval_df)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at 

(2970,
 defaultdict(list,
             {'global_step': [594, 1188, 1782, 2000, 2376, 2970],
              'train_loss': [0.12244381755590439,
               0.040643926709890366,
               0.0607752688229084,
               0.09947275370359421,
               0.02721583843231201,
               0.06510569900274277],
              'eval_loss': [0.2246865946674507,
               0.20709883887945807,
               0.21533899931174597,
               0.23937824150178608,
               0.23549677828489335,
               0.24733661077666602],
              'precision': [0.7406593406593407,
               0.7509043927648579,
               0.7494939271255061,
               0.7552515876893015,
               0.7767676767676768,
               0.7575169738118331],
              'recall': [0.6061151079136691,
               0.6533273381294964,
               0.6659172661870504,
               0.6951438848920863,
               0.6915467625899281,
               0.7023381294964028],
   

In [9]:
result, model_outputs, y_pred = model.eval_model(test_df)

2021-12-13 02:55:23 simpletransformers.ner.ner_model INFO:  Converting to features started.
Running Evaluation: 100%|██████████| 186/186 [00:07<00:00, 26.26it/s]
2021-12-13 02:55:36 simpletransformers.ner.ner_model INFO: {'eval_loss': 0.1064013707650555, 'precision': 0.8846860080793243, 'recall': 0.8622047244094488, 'f1_score': 0.8733007069059271}


In [10]:
result

{'eval_loss': 0.1064013707650555,
 'precision': 0.8846860080793243,
 'recall': 0.8622047244094488,
 'f1_score': 0.8733007069059271}

In [11]:
df_x_test=test_df.groupby('sentence_id')['words'].apply(list)
x_test=list(df_x_test)

df_y_test=test_df.groupby('sentence_id')['labels'].apply(list)
y_test=list(df_y_test)

In [12]:
predictions, raw_outputs = model.predict(
    # ["@PolsekAlian PolriBerikan inovasi-inovasi Polri yang memudahkan masyarakat, kegiatan pemolisian masyarakat dan seluruh tindakan Kepolisian dalam menghadapi gangguan Kamtibmas, dan lain-lain - Jenderal Polisi Drs. Listyo Sigit Prabowo, https://t.co/f8GZ6VPqPD. - Kapolri #PolriTVRadioPresisi"],
    ["@ShopeeID Bismillah yok menang ���� SHOPEE @ShopeeID #44ShopeeMamamoo #ShopeexMAMAMOO #AdaMamamoodiShopee"], 
    ["@OPPOIndonesia ✍Butuh memori besar 128GB di OPPO A54 buat simpan file-file tugas kuliah dan testimoni online shop -ku ��Semoga oppo A54 bisa menunjang penghasilan keluargaku �� @OPPOIndonesia @nisa_rkt @Bebheey @kus_ica @Erna19_ @Saftriyuni #OPPOA54 https://t.co/bMYIiRb4QF �� 2.921"]
        )

2021-12-13 02:55:36 simpletransformers.ner.ner_model INFO:  Converting to features started.
Running Prediction: 100%|██████████| 1/1 [00:00<00:00, 27.02it/s]


In [13]:
predictions

[[{'@ShopeeID': 'O'},
  {'Bismillah': 'O'},
  {'yok': 'O'},
  {'menang': 'O'},
  {'����': 'O'},
  {'SHOPEE': 'B-PROD'},
  {'@ShopeeID': 'O'},
  {'#44ShopeeMamamoo': 'O'},
  {'#ShopeexMAMAMOO': 'O'},
  {'#AdaMamamoodiShopee': 'O'}]]

In [16]:
evaluator = Evaluator(y_test, y_pred, tags=['LOC', 'PER', 'ORG', 'PROD', 'WA', 'EV'], loader="list")

In [17]:
results, results_per_tag = evaluator.evaluate()

In [20]:
results['partial']

{'correct': 2479,
 'incorrect': 0,
 'partial': 82,
 'missed': 296,
 'spurious': 162,
 'possible': 2857,
 'actual': 2723,
 'precision': 0.9254498714652957,
 'recall': 0.8820441022051102,
 'f1': 0.9032258064516129}

In [19]:
results['exact']

{'correct': 2479,
 'incorrect': 82,
 'partial': 0,
 'missed': 296,
 'spurious': 162,
 'possible': 2857,
 'actual': 2723,
 'precision': 0.9103929489533603,
 'recall': 0.8676933846692335,
 'f1': 0.8885304659498208}

In [ ]:
import re

temp_l_l=list()
for i in range(len(y_test)):
    temp_l=list()
    print(x_test[i])
    print(len(x_test[i]))
    print("y_test       ",len(y_test[i]),y_test[i])
    print("y_pred       ",len(y_pred[i]),y_pred[i])
    for j in range(len(y_pred[i])):
        temp_l.append((x_test[i][j],y_test[i][j],y_pred[i][j]))
    temp_l_l.append(temp_l)
    print()

In [ ]:
with open("./results_pred.txt", "w") as f:
    str_results = repr(y_pred)
    f.write(str_results)

In [ ]:
with open("./results_test_pred.txt", "w",encoding="utf-8") as f:
    str_results = repr(temp_l_l)
    f.write(str_results)

In [15]:
with open("./results_1_batch.txt", "w") as f:
    str_results = repr(results)
    f.write(str_results)

In [16]:
with open("./results_per_tag_1_batch.txt", "w") as f:
    str_results_per_tag = repr(results_per_tag)
    f.write(str_results_per_tag)